# 1. Startup Application and Internal Memory 

In [1]:
!pip install -r requirements.txt

In [ ]:
import pickle
import pandas as pd
import hopsworks
import os

from keys import TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3
from TomTomAPI import get_traffic_map, plot_traffic_map, get_grid_from_map, get_traffic_map_from_grid, get_grid_from_map_alt
from HolidaysnWeather import get_weather, is_holiday

In [ ]:
coordinates = 59.34318, 18.05141 # Stockholm near Odenplan
radius = 1000
zoom = 20

TOMTOM_keys = [TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3]

### 1.1. Create GRID for extracting traffic data

In [ ]:
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map = get_traffic_map(TOMTOM_keys[i], coordinates, radius, zoom = zoom)
        success = True
    except:
        print('Failed with key:', i)
        i += 1

In [ ]:
print(len(traffic_map))
traffic_map = set([str(s) for s in traffic_map])
traffic_map = [eval(s) for s in list(traffic_map)]
print(len(traffic_map))

plot_traffic_map(traffic_map)

In [ ]:
grid = get_grid_from_map_alt(traffic_map)

with open('variables/grid.pickle', 'wb') as file:
    # Serialize and write the variable to the file
    pickle.dump(grid, file)

### 1.2. Collect traffic data

In [ ]:
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map2 = get_traffic_map_from_grid(TOMTOM_keys[i],grid, zoom = zoom)
        success = True
    except:
        print('Failed with key:', i)
        i += 1

print(len(traffic_map2))
traffic_map2 = set([str(s) for s in traffic_map2])
traffic_map2 = [eval(s) for s in list(traffic_map2)]
print(len(traffic_map2))

plot_traffic_map(traffic_map2, name = 'figures/traffic_map2.png')

In [ ]:
from shapely.geometry import LineString
import datetime

traffic_df = pd.DataFrame(traffic_map)
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: LineString([(point['longitude'], point['latitude']) for point in x['coordinate']]))
traffic_df = traffic_df.drop_duplicates()

today = datetime.datetime.now()
dates = [today for i in range(traffic_df.shape[0])]
multi_index = pd.MultiIndex.from_arrays(
    [dates,traffic_df['frc'], traffic_df['coordinates']],  # Pass as a list of arrays
    names=['date','frc', 'coordinates']  # Assign level names
)
traffic_df.index = multi_index

traffic_df = traffic_df.drop(columns = ['@version','frc', 'coordinates'])

print(traffic_df.shape)
traffic_df.head(1)

### 1.3. Collect weather data

In [ ]:
weather_df = get_weather(coordinates)
weather_df

### 1.4. Collect weather data

In [ ]:
holiday_status = is_holiday()
weather_df['holiday_status'] = holiday_status
weather_df

### 1.5. Joining data and uploading to Hopsworks

In [ ]:
# Get the API key from GitHub Secrets
# HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')
with open('hopsworks-api-key.txt','r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

# Get AQI API KEY from secrets of hopsworks
project = hopsworks.login(project="ID2223LAB1KTH")

In [ ]:

# Initialize the feature store

fs = project.get_feature_store()

# Define and insert the Air Quality Feature Group
traffic_fg = fs.get_or_create_feature_group(
    name="stockholm_traffic",
    version=1,
    description="Traffic data of odenplan : coordinates currentSpeed	freeFlowSpeed	currentTravelTime	freeFlowTravelTime	confidence	roadClosure",
    primary_key=["date"],  
    event_time="date" 
)

traffic_fg.insert(traffic_df)

# Define and insert the Air Quality Feature Group
weather_fg = fs.get_or_create_feature_group(
    name="stockholm_weather&holiday",
    version=1,
    description="Weather data for Stockholm including temperature, precipitation, wind speed and holiday",
    primary_key=["date"],  
    event_time="date" 
)

weather_fg.insert(weather_df)